In [1]:
!pip install empath
!pip install vaderSentiment
!pip install plydata

from plydata import *
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 

from collections import defaultdict, Counter
from empath import Empath
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer


%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files
from google.colab import drive
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from joblib import load
import pickle
import random
from itertools import tee, islice
import re

# Numba
from numba import jit, njit, vectorize, cuda

# Mount drive
drive.mount('/content/drive', force_remount=True)

     |████████████████████████████████| 61kB 4.4MB/s 
  Created wheel for empath: filename=empath-0.89-cp36-none-any.whl size=57824 sha256=9ab2dc2e74f8b0dd8611c1f078d050dea7e89ef757fe1ff5dced141962428646
  Stored in directory: /root/.cache/pip/wheels/84/ea/2f/2bc54d4f9985ce61753ebc5b00cb2df51d855589267c667308
Successfully built empath
     |████████████████████████████████| 133kB 9.0MB/s 
     |████████████████████████████████| 307kB 8.8MB/s 
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
TensorFlow 1.x selected.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on fu

In [0]:
# Set up for numba
os.environ['NUMBAPRO_LIBDEVICE'] = "/usr/local/cuda-10.1/nvvm/libdevice"
os.environ['NUMBAPRO_NVVM'] = "/usr/local/cuda-10.1/nvvm/lib64/libnvvm.so"

# Import and Load Trained GPT2 Model

In [3]:
# Copy checkpoint
gpt2.copy_checkpoint_from_gdrive(run_name='run1')

# Start session and load model
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name='run1', multi_gpu=True)

# Import scaler
scaler = pickle.load(open('/content/drive/My Drive/5_datascience/2_context_chatbot/data/models/scaler.p', 'rb'))

Loading checkpoint checkpoint/run1/model-1000
INFO:tensorflow:Restoring parameters from checkpoint/run1/model-1000


In [0]:
# calc_semantic_similarity(test_df.loc[0])

In [0]:
# Calculate ngrams as a mechansim for filtering out poor responses, which oftentimes repeated phrases.
def calc_trigrams(text):
    text = text.lower()
    # Tokenize the input text
    tokens = nltk.word_tokenize(text)
    
    # Create trigrams
    tgs = nltk.trigrams(tokens)

    # Compute frequency distribution for all the ngrams in the text
    fdist = nltk.FreqDist(tgs)
    
    return(fdist)

def trigram_present(ngram_fdist):
    for k,v in ngram_fdist.items():
        if v > 1:
            return True
    return False

def max_tgram_freq(ngram_fdist):
    max_value = 0
    for k,v in ngram_fdist.items():
        if v > max_value:
            max_value = v
    return v


In [0]:
# test_grams = calc_trigrams("Jason is the best in the world. Just in case you think different, fuck you.")
# max_tgram_freq(test_grams)

In [0]:
def add_prefix_to_dict_keys(dict, prefix):
    return {f'{prefix}_{k}': v for k, v in dict.items()}

# calc similarity was below here 
def calc_semantic_similarity(prompt_response_row_df):
    x = prompt_response_row_df['prompt']
    y = prompt_response_row_df['response']

    # tokenization first
    x_list = word_tokenize(x)
    y_list = word_tokenize(y)

    # Check for stop words
    sw = stopwords.words('english')  
    l1 = list()
    l2 = list()

    # Remove stop words from string
    x_set = {w for w in x_list if not w in sw}  
    y_set = {w for w in y_list if not w in sw}  


    # form a set containing keywords of both strings  
    rvector = x_set.union(y_set) 
    for w in rvector: 
        if w in x_set: l1.append(1) # create a vector 
        else: l1.append(0) 
        if w in y_set: l2.append(1) 
        else: l2.append(0)  
    
    l1_arr = np.array(l1)
    l2_arr = np.array(l2)

    return calc_cosine(l1_arr, l2_arr, len(rvector))


@njit(cache=True)
def calc_cosine(l1_arr, l2_arr, len_rvector):

    c = 0
    for i in range(len_rvector): 
        c+= l1_arr[i]*l2_arr[i] 
    cosine = c / float((np.sum(l1_arr)*np.sum(l2_arr))**0.5) 
    return cosine


class SeanceScorer():

    def __init__(self):
        # add in inquiererbasic.txt
        with open('/content/drive/My Drive/5_datascience/2_context_chatbot/data/inputs/seance/inquirerbasic.txt', 'r') as content_file:
            content = content_file.read()

        split_by_metric = content.split('\n')

        self.seance_dict = {}
        for line in split_by_metric:
            word_list = line.split('\t')
            self.seance_dict[f'{word_list[0]}'] = word_list[1:]

        # add in negative_words.txt
        with open('/content/drive/My Drive/5_datascience/2_context_chatbot/data/inputs/seance/negative_words.txt', 'r') as content_file:
            content = content_file.read()

        self.seance_dict['negative_words'] = content.split('\n')

        # add in positive_words.txt
        with open('/content/drive/My Drive/5_datascience/2_context_chatbot/data/inputs/seance/positive_words.txt', 'r') as content_file:
            content = content_file.read()

        self.seance_dict['positive_words'] = content.split('\n')
    

    def calc_seance_scores(self, text_to_score, type_scored):
        scores_dict = {}
        for dict_key in self.seance_dict.keys():
            cur_list = self.seance_dict[dict_key] # The list of words associated with the seance feature
            counted = Counter(text_to_score.split())
            common_words = set(cur_list).intersection(counted)
            count_common = sum(counted[wrd] for wrd in set(cur_list))
            scores_dict[f'{type_scored}_s_{dict_key}'] = count_common
        return(scores_dict)

def choose_optimal_reponse(responses_list):
    pass

# myround stuff was here 
def myround(x, base=5):
    return base * round(x/base)


def score_senti_prompts_responses(prompt_scores_df, seance_scorer, lexicon, analyzer, take_out_speaker_in_prompt=True):
    all_scores_list = list()
    for i in range(len(prompt_scores_df)):
        prompt = prompt_scores_df['prompt'][i]
        if take_out_speaker_in_prompt:
            prompt = prompt.replace('CLIENT:', '')
            prompt = prompt.replace('THERAPIST:', '')
        response = prompt_scores_df['response'][i]
        seance_input_scores = pd.DataFrame(seance_scorer.calc_seance_scores(prompt, type_scored='prompt'), index=[i])
        seance_output_scores = pd.DataFrame(seance_scorer.calc_seance_scores(response, type_scored='response'), index=[i])
        empath_input_scores = pd.DataFrame(add_prefix_to_dict_keys(lexicon.analyze(prompt), prefix='prompt_e'), index=[i])
        empath_output_scores = pd.DataFrame(add_prefix_to_dict_keys(lexicon.analyze(response), prefix='response_e'), index=[i])
        vader_input_scores = pd.DataFrame(add_prefix_to_dict_keys(analyzer.polarity_scores(prompt), prefix='prompt_v'), index=[i])
        vader_output_scores = pd.DataFrame(add_prefix_to_dict_keys(analyzer.polarity_scores(response), prefix='response_v'), index=[i])

        all_scores_list.append(pd.concat([seance_input_scores, seance_output_scores, empath_input_scores, empath_output_scores, vader_input_scores, vader_output_scores], axis=1))

    all_scores = pd.concat(all_scores_list)
    return(all_scores)


def score_generated_responses(responses_df):
    # initializes three of the scorers
    seance_scorer = SeanceScorer() # from custom made seance scorer
    lexicon = Empath() # from empath library
    analyzer = SentimentIntensityAnalyzer()

    all_scores_df = score_senti_prompts_responses(responses_df, seance_scorer, lexicon, analyzer)
    all_scores_df['similarity_score'] = responses_df.apply(calc_semantic_similarity, axis=1)
    return all_scores_df


def generate_unique_responses_for_prompt(input_text, num_responses, verbose=0):
    """ Generates a dataframe with repeated input texts, num_responses outputs, and temp values.

    Keyword arguments:
    input_text -- The string that is inputted into the gp2 text generative model.
    num_responses -- Desired number of unique responses. 

    """
    temp = .4
    data_dict_list = list()

    # Generate text from the combined input text
    while len(data_dict_list) < num_responses:
        nsamples = num_responses - len(data_dict_list)
        nsamples = myround(nsamples, 20) + 20
        generated_texts = gpt2.generate(sess,
                  length=120,
                  temperature=temp,
                  prefix=input_text,
                  nsamples=nsamples,
                  batch_size=20,
                  return_as_list=True
                  )

        # check if the outputs meet the requirements. 
        for k in range(len(generated_texts)):
        
            # Check if the response has a 'THERAPIST: in the generated response' 
            if len(generated_texts[k].split(input_text)[1].split('THERAPIST:')) > 0:
                sample_response = generated_texts[k].split(input_text)[1].split('CLIENT:')[0].strip()
                tgram_fdis = calc_trigrams(sample_response)

                # Check if the length of the possible response is greater than 1 
                # and add the sample response 
                if len(sample_response) > 1 and len(data_dict_list) < num_responses and not trigram_present(tgram_fdis):
                    # Checks for if the sample response has already been inputted into the list. 
                    if not any(d['response'] == sample_response for d in data_dict_list): 
                        data_dict = dict()
                        data_dict['temp'] = temp
                        data_dict['prompt'] = input_text
                        data_dict['response'] = sample_response

                        data_dict_list.append(data_dict)


        temp += .05 # randomness parameter

        if verbose > 0:
            print(f"{len(data_dict_list)} out of {num_responses} responses have been generated.")

    data_df = pd.DataFrame(data_dict_list)
    return(data_df)


def interact_with_bot(moving_window_used, moving_window=1, min_len_response=15, remove_responses=False, num_samples = 1, verbose=0):

    # initialize inputs and chatbot reponses
    input_text = ""
    chatbot_response = ""
    i = 0
    oldtext = ""
    temp = .8 # what does temp do?

    # Initialize lists to store the prompts and responses.
    client_prompts = list()
    therapist_responses = list()
    interaction_history = list()

    # Initialize the filtering model
    rf_model = load('/content/drive/My Drive/5_datascience/2_context_chatbot/data/models/default_rf.joblib')

    while input_text!="ABORT" or input_text!= "SAVE" :
        i = i + 1        
        if i==1:

            print("Can you tell me what you want to talk about today? It helps if you say at least a few sentences for context.")
        input_text = input("\n")
        client_prompts.append(input_text) # adds the user input to prompts list

        if input_text=="ABORT":
            break
        elif input_text=="SAVE":
            save_df = pd.DataFrame(interaction_history)
            return(save_df)

        if(remove_responses == False):
        # Filter the input text so that it fits into the moving window
            if(len(client_prompts) <= moving_window):
                combined_text = ""
                for j in range(len(client_prompts)):
                    if j != 0:
                        combined_text = combined_text + therapist_responses[j - 1]

                    combined_text = combined_text + "\nCLIENT: " + client_prompts[j] + "\nTHERAPIST:" 
            else: 
                moving_window_used = True
                combined_text = ""
                prompts_subset = client_prompts[-moving_window:]
                responses_subset = therapist_responses[-moving_window + 1:]

                for j in range(len(prompts_subset)):
                    if j != 0:
                        combined_text = combined_text + responses_subset[j - 1]
                    combined_text = combined_text + "\nCLIENT: " + prompts_subset[j] + "\nTHERAPIST:" 

        else: 
            combined_text = "Client: " + ' '.join(client_prompts[-moving_window:])

        combined_text = combined_text.strip()
        if verbose > 0:
            print(f'\nGPT2 input text is ... \n{combined_text} \n')


        # Generate unique responses for a given input 
        # change input_text=combined_text to use moving window
        responses_df = generate_unique_responses_for_prompt(input_text= combined_text, num_responses=100, verbose=verbose)

        # Score the unique responses and use a random forest model to estimate how useful a given response is.
        X = pd.concat([score_generated_responses(responses_df), responses_df['temp']], axis=1)
        X = X.reindex(sorted(X.columns), axis=1) 
        X_scaled = scaler.transform(X)
        y_pred = pd.Series(rf_model.predict(X_scaled))
        y_pred = y_pred.rename('y_pred')
        responses_preds_df = pd.concat([responses_df, y_pred], axis=1)
        responses_preds_df = responses_preds_df.sort_values(by='y_pred', ascending=False)
        best_response = responses_preds_df.iloc[0]['response']
        interaction_history.append(responses_preds_df.loc[0])
        therapist_responses.append(best_response)
        if verbose > 0:
            print(f"The top 5 potential responses are: \n")
            top_5_df = responses_preds_df.head(5)
            for index, row in top_5_df.iterrows():
                print(f"{row['y_pred']}: {row['response']}")
    
        print("\nTherapist Bot: "+ best_response) #change 1 to i if the prior is re-insurted 

     

In [0]:
saved_history = interact_with_bot(verbose=1, moving_window_used=True)
saved_history

Can you tell me what you want to talk about today? It helps if you say at least a few sentences for context.

I am very hungry right now. 

GPT2 input text is ... 
CLIENT: I am very hungry right now. 
THERAPIST: 

37 out of 100 responses have been generated.
56 out of 100 responses have been generated.
70 out of 100 responses have been generated.
91 out of 100 responses have been generated.
97 out of 100 responses have been generated.
100 out of 100 responses have been generated.
The top 5 potential responses are: 

3.95: I wonder if there's a way you can kind of tease them apart a little bit.
3.66: I wonder if it's hard to talk about it.
3.39: I wonder if it's a little bit about the way you're feeling right now.
3.345: Yeah. I mean I think that's part of what we're talking about is not just your hunger, but your fear that you could get something for her.
3.31: That's good.

Therapist Bot: I wonder if there's a way you can kind of tease them apart a little bit.

Life is super dull and 

In [0]:
test_string = 'CLIENT: Hello I am always feeling so tired. What a waste of life.\nTHERAPIST:'
test_df = generate_unique_responses_for_prompt(test_string, num_responses=20, verbose=1)
X = pd.concat([score_generated_responses(test_df), test_df['temp']], axis=1)
X = X.reindex(sorted(X.columns), axis=1)
X = scaler.transform(X)

16 out of 20 responses have been generated.
20 out of 20 responses have been generated.


In [0]:
test_list = []
test_list.append(test_df.loc[0])
test_list.append(test_df.loc[0])

test_result = pd.DataFrame(test_list)

In [0]:
test_df.loc[0]

temp                                                      0.4
prompt      CLIENT: Hello I am always feeling so tired. Wh...
response                                It's a waste of life.
Name: 0, dtype: object

In [0]:
test_result

,temp,prompt,response
0,0.4,CLIENT: Hello I am always feeling so tired. Wh...,It's a waste of life.
0,0.4,CLIENT: Hello I am always feeling so tired. Wh...,It's a waste of life.


%timeit myround_njit(random.randint(1,100))

In [0]:
test_string = 'CLIENT: Hello I am always feeling so tired. What a waste of life.\nTHERAPIST:'
test_df = generate_unique_responses_for_prompt(test_string, num_responses=20, verbose=1)
X = pd.concat([score_generated_responses(test_df), test_df['temp']], axis=1)
X = X.reindex(sorted(X.columns), axis=1)
X = scaler.transform(X)

rf_model = load('/content/drive/My Drive/5_datascience/2_context_chatbot/data/models/default_rf.joblib')
y_pred = pd.Series(predict_and_rank(X, rf_model))
y_pred = y_pred.rename('y_pred')
responses_preds_df = pd.concat([test_df, y_pred], axis=1)
responses_preds_df = responses_preds_df.sort_values(by='y_pred', ascending=False)


20 out of 20 responses have been generated.


NameError: ignored

In [0]:
responses_preds_df = responses_preds_df.sort_values(by='y_pred', ascending=False)
best_response = responses_preds_df.iloc[0]['response']

In [0]:
best_response

"You're a real person."

array([[0.        , 0.07449132, 0.        , ..., 0.00499092, 0.03583965,
        0.02979653],
       [0.        , 0.04646402, 0.        , ..., 0.00218381, 0.00948443,
        0.01858561],
       [0.        , 0.07037979, 0.        , ..., 0.01309064, 0.01225155,
        0.02815192],
       ...,
       [0.        , 0.11327224, 0.        , ..., 0.        , 0.0231216 ,
        0.04530889],
       [0.        , 0.02337794, 0.        , ..., 0.00032729, 0.00441801,
        0.00935118],
       [0.        , 0.0897509 , 0.        , ..., 0.        , 0.01495848,
        0.03590036]])

In [0]:
# Split the data into features and labels. 
X = all_scored_full >> select('-input','-output_list', '-NCJ_Rating')
X = X.reindex(sorted(X.columns), axis=1)
y = all_scored_full['NCJ_Rating']

# Normalize the features
X = normalize(X)